In [40]:
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

In [57]:
creds = {
    'DB_HOST': 'liradb.compute.dtu.dk',
    'DB_PORT': 5435,
    'DB_USER': 'guest',
    'DB_PASS': 'AvpF7r8b',
    'DB_NAME': 'postgres',
    'SSH_HOST': 'thinlinc.compute.dtu.dk',
    'SSH_USER': 's174420',
    'SSH_PASS': open('creds.json').readline()
}

In [65]:
print('Connecting to the SSH Tunnel...')

ssh_tunnel = SSHTunnelForwarder(
    ssh_address_or_host=(creds['SSH_HOST'], 22),
    ssh_username=creds["SSH_USER"],
    ssh_password=creds['SSH_PASS'],
    remote_bind_address=(creds["DB_HOST"], creds['DB_PORT']),
    allow_agent=False
)

ssh_tunnel.start()

print(ssh_tunnel.local_bind_port)
print('Connecting to the PostgreSQL Database...')

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
    host=creds["DB_HOST"],
    port=ssh_tunnel.local_bind_port,
    user=creds["DB_USER"],
    password=creds["DB_PASS"],
    db=creds["DB_NAME"]
))

engine.connect()

Connecting to the SSH Tunnel...


2022-02-22 14:55:01,017| ERROR   | Could not connect to gateway thinlinc.compute.dtu.dk:22 : Unable to connect to thinlinc.compute.dtu.dk: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [6]:
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
    host=creds["HOST"],
    port=creds['PORT'],
    user=creds["USER"],
    password=creds["PW"],
    db=creds["DB"]
))

In [8]:
engine.connect()

OperationalError: (psycopg2.OperationalError) connection to server at "liradbdev.compute.dtu.dk" (130.225.69.110), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
engine.